# GRS-34806 MGI Project
## **1) Download the data**
This template downloads the UCM data (both mono and multi-labels) in your local Colab environment. 

Write the notebook in such a way that it fully runs from start to end without further intervention  
(i.e. do not change the directory structure manually in the mean time).

Good luck!

In [ ]:
!pip install d2l==0.16.1

In [ ]:
import os
import zipfile

! git clone https://git.wur.nl/lobry001/ucmdata.git
os.chdir('ucmdata')

with zipfile.ZipFile('UCMerced_LandUse.zip', 'r') as zip_ref:
    zip_ref.extractall('UCMImages')

!mv UCMImages/UCMerced_LandUse/Images .
!rm -rf UCMImages README.md  UCMerced_LandUse.zip
!ls

UCM_images_path = "Images/"
Multilabels_path = "LandUse_Multilabeled.txt"

##**2) Preprocessing the data**

#### Extract labels and prepare classes data


In [ ]:
# Import pandas and load multilabels dataset
import pandas as pd
df = pd.read_csv(Multilabels_path, sep="\t")

# Extract labels for each image
labels = df[df.columns[0]].str.replace('\d+', '')
labels = pd.DataFrame(labels)

# List the 21 classes
classes = df[df.columns[0]].str.replace('\d+', '').unique()

# Assign values per label
for i in range(len(classes)):
  labels.loc[labels[labels.columns[0]].str.contains(classes[i]), "value"] = i

# Convert floats to integer
labels["value"] = labels["value"].astype(int)

#### Split function and split image numbers

In [ ]:
# Generate list with all image numbers
img_numbers = ["%.2d" % i for i in range(100)]

# Define split function to split all images in even and odd numbers (1050/1050)
def split_numbers(numbers): 
   divisionlist = [] 
   restlist = [] 
   for i in range(len(numbers)): 
      if (i % 5 == 0): 
         divisionlist.append(numbers[i]) 
      else: 
         restlist.append(numbers[i]) 
   return (restlist, divisionlist)

# Assign even and odd image numbers for train and test split
train_num, test_num = split_numbers(img_numbers)

#### Split label numbers

In [ ]:
# Generate index list for all image labels
label_numbers = [i for i in range(2100)]

# Assign even and odd image numbers for train and test split (1050/1050)
train_label_num, test_label_num = split_numbers(label_numbers)

# Get corresponding label function
def getLabel (labels, numbers):
  img_label = []
  for i in numbers:
    img_label.append(labels["value"][i])
  return(img_label)

# Select training and test labels
train_labels = getLabel(labels, train_label_num)
test_labels = getLabel(labels, test_label_num)

#### Function to put images in tensors

In [ ]:
import os
from skimage import io
import numpy as np
import torch
import torchvision.transforms as T

def load_data (classes, ids):

  image_ids = ids
  imgs = []
  # Assign values per label
  for i in range(len(classes)):
    label = classes[i]
    img_folder = os.path.join(UCM_images_path, label + "/") #Images/agricultural/
    
    #This will convert the numpy array to a tensor
    conversion = T.ToTensor()

    for img_index in image_ids:
      
      #Load the tile and the corresponding ground truth.
      img = io.imread(os.path.join(img_folder, label + str(img_index) + '.tif')).astype(np.float32)/255 #Images/agricultural/agricultural01.tif
      imgs.append(conversion(img))
      
      #print("Working on image " + classes[i] + str(img_index))
      #print("\nimgs has " + str(len(imgs)) + " images")
      #print("\nThe first image of imgs has " + str(imgs[0].shape) +"\n")
      #print(len(imgs))

  return (imgs)

#### Prepare training and test images

In [ ]:
train_data = load_data(classes, train_num)
test_data = load_data(classes, test_num)

##3) Define dataset class and load the data

In [ ]:
from torch.utils.data import Dataset

class UCMDataset(Dataset):
  def __init__(self, images, labels):
    self.imgs = images
    self.labs = labels
    self.resize = T.Resize((256, 256))

  def __len__(self):
    return len(self.imgs)

  def __getitem__(self, idx):
    img = self.resize(self.imgs[idx].float())
    lab = self.labs[idx]

    return img, lab

#### Prepare training and test dataset

In [ ]:
#augs = torchvision.transforms.Compose([
    #torchvision.transforms.RandomHorizontalFlip(), color_aug, shape_aug])
#first line add 'transform=augs'
#dataset = torchvision.datasets.CIFAR10(root="../data", train=is_train,
                                           #transform=augs, download=True)

training_dataset = UCMDataset(train_data, train_labels)
test_dataset = UCMDataset(test_data, test_labels)

#### Dataloader

In [ ]:
batch_size = 128

train_iter = torch.utils.data.DataLoader(dataset = training_dataset, batch_size = batch_size, shuffle = True)
test_iter = torch.utils.data.DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle = False)

In [ ]:
iterable_train_loader = enumerate(train_iter)
iterable_test_loader = enumerate(test_iter)

#### Visualise image with label. Check if they correspond

In [ ]:
X, y = next(iter(train_iter))

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

sample_id = 0
image = X[sample_id].permute(1,2,0)
imgplot = plt.imshow(image)
print("label = " + str(classes[y[sample_id]]))

## **4) Create model classes**

#### Basic model

In [ ]:
import torch
from torch import nn
#Basic model
class Hypercolumns1(nn.Module):
  def __init__(self):
    super(Hypercolumns1, self).__init__()

    self.conv1 = nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=1)
    self.conv2 = nn.Conv2d(96, 256, kernel_size=5, padding=2)
    self.conv3 = nn.Conv2d(256, 384, kernel_size=3, padding=1)
    self.conv4 = nn.Conv2d(384, 384, kernel_size=3, padding=1)
    self.conv5 = nn.Conv2d(384, 256, kernel_size=3, padding=1)

    self.lin1 = nn.Linear(9216, 4096)
    self.lin2 = nn.Linear(4096, 4096)
    self.lin3 = nn.Linear(4096, 21)

    self.pool = nn.MaxPool2d(kernel_size=3, stride=2)

    self.relu = nn.ReLU()

    self.flatten = nn.Flatten()

    self.dropout = nn.Dropout(p=0.5)

  def forward(self, x):
    x = self.conv1(x)
    x = self.relu(x)
    x = self.pool(x)
    # print('Conv1 shape: '+str(x.shape))

    x = self.conv2(x)
    x = self.relu(x)
    x = self.pool(x)
    # print('Conv2 shape: '+str(x.shape))

    x = self.conv3(x)
    x = self.relu(x)
    # print('Conv3 shape: '+str(x.shape))

    x = self.conv4(x)
    x = self.relu(x)
    # print('Conv4 shape: '+str(x.shape))

    x = self.conv5(x)
    x = self.relu(x)
    x = self.pool(x)
    x = self.flatten(x)
    # print('Conv5 shape: '+str(x.shape))

    x = self.lin1(x)
    x = self.relu(x)
    x = self.dropout(x)
    # print('Lin1 shape: '+str(x.shape))

    x = self.lin2(x)
    x = self.relu(x)
    x = self.dropout(x)
    # print('Lin2 shape: '+str(x.shape))

    x = self.lin3(x)
    # print('Lin3 shape: '+str(x.shape))

    return x

#### Model with batch normalization before activation function

In [ ]:
import torch
from torch import nn
#Batch norm before relu
class Hypercolumns2(nn.Module):
  def __init__(self):
    super(Hypercolumns2, self).__init__()

    self.conv1 = nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=1)
    self.conv2 = nn.Conv2d(96, 256, kernel_size=5, padding=2)
    self.conv3 = nn.Conv2d(256, 384, kernel_size=3, padding=1)
    self.conv4 = nn.Conv2d(384, 384, kernel_size=3, padding=1)
    self.conv5 = nn.Conv2d(384, 256, kernel_size=3, padding=1)

    self.bn1 = nn.BatchNorm2d(96)
    self.bn2 = nn.BatchNorm2d(256)
    self.bn3 = nn.BatchNorm2d(384)

    self.lin1 = nn.Linear(9216, 4096)
    self.lin2 = nn.Linear(4096, 4096)
    self.lin3 = nn.Linear(4096, 21)

    self.pool = nn.MaxPool2d(kernel_size=3, stride=2)

    self.relu = nn.ReLU()

    self.flatten = nn.Flatten()

    self.dropout = nn.Dropout(p=0.5)

  def forward(self, x):
    x = self.conv1(x)
    x = self.bn1(x)
    x = self.relu(x)
    x = self.pool(x)

    x = self.conv2(x)
    x = self.bn2(x)
    x = self.relu(x)
    x = self.pool(x)

    x = self.conv3(x)
    x = self.bn3(x)
    x = self.relu(x)

    x = self.conv4(x)
    x = self.bn3(x)
    x = self.relu(x)

    x = self.conv5(x)
    x = self.bn2(x)
    x = self.relu(x)
    x = self.pool(x)
    x = self.flatten(x)

    x = self.lin1(x)
    x = self.relu(x)
    x = self.dropout(x)

    x = self.lin2(x)
    x = self.relu(x)
    x = self.dropout(x)

    x = self.lin3(x)

    return x

#### Model with batch normalization after activation function

In [ ]:
import torch
from torch import nn
#Batch norm after relu
class Hypercolumns3(nn.Module):
  def __init__(self):
    super(Hypercolumns3, self).__init__()

    self.conv1 = nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=1)
    self.conv2 = nn.Conv2d(96, 256, kernel_size=5, padding=2)
    self.conv3 = nn.Conv2d(256, 384, kernel_size=3, padding=1)
    self.conv4 = nn.Conv2d(384, 384, kernel_size=3, padding=1)
    self.conv5 = nn.Conv2d(384, 256, kernel_size=3, padding=1)

    self.bn1 = nn.BatchNorm2d(96)
    self.bn2 = nn.BatchNorm2d(256)
    self.bn3 = nn.BatchNorm2d(384)

    self.lin1 = nn.Linear(9216, 4096)
    self.lin2 = nn.Linear(4096, 4096)
    self.lin3 = nn.Linear(4096, 21)

    self.pool = nn.MaxPool2d(kernel_size=3, stride=2)

    self.relu = nn.ReLU()

    self.flatten = nn.Flatten()

    self.dropout = nn.Dropout(p=0.5)

  def forward(self, x):
    x = self.conv1(x)
    x = self.relu(x)
    x = self.bn1(x)
    x = self.pool(x)

    x = self.conv2(x)
    x = self.relu(x)
    x = self.bn2(x)
    x = self.pool(x)

    x = self.conv3(x)
    x = self.relu(x)
    x = self.bn3(x)

    x = self.conv4(x)
    x = self.relu(x)
    x = self.bn3(x)

    x = self.conv5(x)
    x = self.relu(x)
    x = self.bn2(x)
    x = self.pool(x)
    x = self.flatten(x)

    x = self.lin1(x)
    x = self.relu(x)
    x = self.dropout(x)

    x = self.lin2(x)
    x = self.relu(x)
    x = self.dropout(x)

    x = self.lin3(x)

    return x

#### Define all three models

In [ ]:
net_basic = Hypercolumns1()
net_bn_bf = Hypercolumns2()
net_bn_af = Hypercolumns3()

#### Check the shapes

In [ ]:
batch = next(iterable_train_loader)
#[1][0] = images [1][1] = labels
output = net_basic(batch[1][0])
print(output.shape)

torch.Size([128, 21])


## **5) Prepare funcions and classes for training**

In [ ]:
#Set device
def try_gpu(i=0):
    """Return gpu(i) if exists, otherwise return cpu()."""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

In [ ]:
def accuracy(y_hat, y):
    """Compute the number of correct predictions."""
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
        y_hat = d2l.argmax(y_hat, axis=1)
    cmp = d2l.astype(y_hat, y.dtype) == y
    return float(d2l.reduce_sum(d2l.astype(cmp, y.dtype)))

In [ ]:
#add the sum over n variables
class Accumulator: 
  def __init__(self, n):
    self.data = [0.0] * n
  def add(self, *args):
    self.data = [a + float(b) for a, b in zip(self.data, args)]
  def reset(self):
    self.data = [0.0] * len(self.data)
  def __getitem__(self, idx):
    return self.data[idx]

In [ ]:
def evaluate_accuracy_gpu(net, data_iter, device=None):
    """Compute the accuracy for a model on a dataset using a GPU."""
    if isinstance(net, torch.nn.Module):
        net.eval()  # Set the model to evaluation mode
        if not device:
            device = next(iter(net.parameters())).device
    # No. of correct predictions, no. of predictions
    metric = Accumulator(2)
    for X, y in data_iter:
        if isinstance(X, list):
            # Required for BERT Fine-tuning (to be covered later)
            X = [x.to(device) for x in X]
        else:
            X = X.to(device)
        y = y.to(device)
        metric.add(accuracy(net(X), y), d2l.size(y))
    return metric[0] / metric[1]


#### Model training function

In [ ]:
from d2l import torch as d2l

def train_model(net, train_iter, test_iter, num_epochs, lr, device):
    """Train a model with a GPU."""
    def init_weights(m):
        if type(m) == nn.Linear or type(m) == nn.Conv2d:
            nn.init.xavier_uniform_(m.weight)

    net.apply(init_weights)
    print('training on', device)
    net.to(device)
    optimizer = torch.optim.SGD(net.parameters(), lr=lr)
    loss = nn.CrossEntropyLoss()
    animator = d2l.Animator(xlabel='epoch', xlim=[1, num_epochs], ylim=[0,1],
                            legend=['train loss', 'train acc', 'test acc'])
    timer, num_batches = d2l.Timer(), len(train_iter)
    for epoch in range(num_epochs):
        # Sum of training loss, sum of training accuracy, no. of examples
        metric = Accumulator(3)
        net.train()
        for i, (X, y) in enumerate(train_iter):
            timer.start()
            optimizer.zero_grad()
            X, y = X.to(device), y.to(device)
            y_hat = net(X)
            l = loss(y_hat, y)
            l.backward()
            optimizer.step()
            with torch.no_grad():
                metric.add(l * X.shape[0], accuracy(y_hat, y), X.shape[0])
            timer.stop()
            train_l = metric[0] / metric[2]
            train_acc = metric[1] / metric[2]
            if (i + 1) % (num_batches // 5) == 0 or i == num_batches - 1:
                animator.add(epoch + (i + 1) / num_batches,
                             (train_l, train_acc, None))
        test_acc = evaluate_accuracy_gpu(net, test_iter)
        animator.add(epoch + 1, (None, None, test_acc))
    print(f'loss {train_l:.3f}, train acc {train_acc:.3f}, '
          f'test acc {test_acc:.3f}')
    print(f'{metric[2] * num_epochs / timer.sum():.1f} examples/sec '
          f'on {str(device)}')

## **6) Train the models**

#### Basic

In [ ]:
# Basic model:
lr, num_epochs = 0.01, 200
train_model(net=net_basic, train_iter=train_iter, test_iter=test_iter, num_epochs=num_epochs, lr=lr, device=try_gpu())

#### Before relu

In [ ]:
# With batch normalization before relu
lr, num_epochs = 0.01, 100
train_model(net=net_bn_bf, train_iter=train_iter, test_iter=test_iter, num_epochs=num_epochs, lr=lr, device=try_gpu())

#### After relu

In [ ]:
# With batch normalization after relu
lr, num_epochs = 0.05, 100
train_model(net=net_bn_af, train_iter=train_iter, test_iter=test_iter, num_epochs=num_epochs, lr=lr, device=try_gpu())

## **7) Visualize accuracy**

#### visualise predictions for a single batch

In [ ]:
batch = next(iterable_test_loader)
pred = net_bn_af(batch[1][0].to(try_gpu()))
fig, axs = plt.subplots(batch_size, 1)
fig.set_size_inches(15,5*batch_size)
for i in range(batch_size):
  axs[i].imshow(batch[1][0].permute(0,2,3,1)[i,:,:,:])
  axs[i].set_title("Label = " + str(classes[batch[1][1][i]]) + "\nPrediction = " + str(classes[pred[i].max(0)[1]]))  

#### Calculate accuracy for a single batch

In [ ]:
# Accuracy
def Calc_Accuracy():
  x = 0
  for i in range(batch_size):
    if str(classes[batch[1][1][i]]) == str(classes[pred[i].max(0)[1]]):
      x = x + 1
  result = str(x / batch_size * 100) + "%"
  return result

In [ ]:
# The test data isn't shuffled, so depending on the classes available in the batch
# It will can give a higher or lower accuracy than expected.

# batch = next(iterable_test_loader)
# pred = net_bn_af(batch[1][0].to(try_gpu()))

Accuracy = "The accuracy = " + Calc_Accuracy()
Accuracy

---
---
## **8) Multilabel classes**

#### Prepare and load data

In [ ]:
# Prepare multilabel dataframe
multicolumns = df[df.columns[1:]]
multicolumns = multicolumns.astype(np.float32)
multicolumns = multicolumns.to_numpy()

In [ ]:
# Get corresponding multilabel function
def getMultiLabel (labels, numbers):
  img_label = []
  for i in numbers:
    img_label.append(multicolumns[i])
  return(img_label)

# Select training and test multilabels
train_multilabels = getMultiLabel(multicolumns, train_label_num)
test_multilabels = getMultiLabel(multicolumns, test_label_num)

In [ ]:
Multi_train = UCMDataset(train_data, train_multilabels)
Multi_test = UCMDataset(test_data, test_multilabels)

In [ ]:
batch_size = 128

Multi_train_iter = torch.utils.data.DataLoader(dataset = Multi_train, batch_size = batch_size, shuffle = True)
Multi_test_iter = torch.utils.data.DataLoader(dataset = Multi_test, batch_size = batch_size, shuffle = False)

#### Basic model for 17 classes

In [ ]:
import torch
from torch import nn
#Basic model multilabel
class Hypercolumns_multi(nn.Module):
  def __init__(self):
    super(Hypercolumns_multi, self).__init__()

    self.conv1 = nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=1)
    self.conv2 = nn.Conv2d(96, 256, kernel_size=5, padding=2)
    self.conv3 = nn.Conv2d(256, 384, kernel_size=3, padding=1)
    self.conv4 = nn.Conv2d(384, 384, kernel_size=3, padding=1)
    self.conv5 = nn.Conv2d(384, 256, kernel_size=3, padding=1)

    self.lin1 = nn.Linear(9216, 4096)
    self.lin2 = nn.Linear(4096, 4096)
    self.lin3 = nn.Linear(4096, 17)

    self.pool = nn.MaxPool2d(kernel_size=3, stride=2)

    self.relu = nn.ReLU()

    self.flatten = nn.Flatten()

    self.dropout = nn.Dropout(p=0.5)

  def forward(self, x):
    x = self.conv1(x)
    x = self.relu(x)
    x = self.pool(x)

    x = self.conv2(x)
    x = self.relu(x)
    x = self.pool(x)

    x = self.conv3(x)
    x = self.relu(x)

    x = self.conv4(x)
    x = self.relu(x)

    x = self.conv5(x)
    x = self.relu(x)
    x = self.pool(x)
    x = self.flatten(x)

    x = self.lin1(x)
    x = self.relu(x)
    x = self.dropout(x)

    x = self.lin2(x)
    x = self.relu(x)
    x = self.dropout(x)

    x = self.lin3(x)

    return x

#### Define model

In [ ]:
net_multi = Hypercolumns_multi()

#### Train model function

In [ ]:
def train_multi_model(net, train_iter, test_iter, num_epochs, lr, device):
    """Train a model with a GPU (defined in Chapter 6)."""
    def init_weights(m):
        if type(m) == nn.Linear or type(m) == nn.Conv2d:
            nn.init.xavier_uniform_(m.weight)

    net.apply(init_weights)
    print('training on', device)
    net.to(device)
    optimizer = torch.optim.SGD(net.parameters(), lr=lr)
    loss = nn.BCELoss()
    animator = d2l.Animator(xlabel='epoch', xlim=[1, num_epochs], ylim=[0,1],
                            legend=['train loss', 'train acc', 'test acc'])
    timer, num_batches = d2l.Timer(), len(train_iter)
    for epoch in range(num_epochs):
        # Sum of training loss, sum of training accuracy, no. of examples
        metric = Accumulator(3)
        net.train()
        for i, (X, y) in enumerate(train_iter):
            timer.start()
            optimizer.zero_grad()
            X, y = X.to(device), y.to(device)
            y_hat = net(X)
            l = loss(y_hat, y)
            l.backward()
            optimizer.step()
            with torch.no_grad():
                metric.add(l * X.shape[0], accuracy(y_hat, y), X.shape[0])
            timer.stop()
            train_l = metric[0] / metric[2]
            train_acc = metric[1] / metric[2]
            if (i + 1) % (num_batches // 5) == 0 or i == num_batches - 1:
                animator.add(epoch + (i + 1) / num_batches,
                             (train_l, train_acc, None))
        test_acc = evaluate_accuracy_gpu(net, test_iter)
        animator.add(epoch + 1, (None, None, test_acc))
    print(f'loss {train_l:.3f}, train acc {train_acc:.3f}, '
          f'test acc {test_acc:.3f}')
    print(f'{metric[2] * num_epochs / timer.sum():.1f} examples/sec '
          f'on {str(device)}')

#### Check the shapes for inconsistenty

In [ ]:
X, y = next(iter(Multi_train_iter))
print(X.shape)
print(y.shape)

torch.Size([128, 3, 256, 256])
torch.Size([128, 17])


In [ ]:
device = try_gpu()
X.to(device)
y_hat = net_multi(X)

In [ ]:
y_hat.shape

torch.Size([128, 17])

#### Train the model

In [ ]:
lr, num_epochs = 0.05, 10
train_multi_model(net=net_multi, train_iter=Multi_train_iter, test_iter=Multi_test_iter, num_epochs=num_epochs, lr=lr, device=try_gpu())

# RuntimeError: The size of tensor a (128) must match the size of tensor b (17) at non-singleton dimension 1
# Both tensors have the same input shape. Check the code blocks above. We couldn't figure out why this was the case